In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from langchain_core.messages import (
    AIMessage,
    AnyMessage,
    HumanMessage,
    RemoveMessage,
    SystemMessage,
    ToolMessage,
)

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv(override=True)
llm = init_chat_model("openai:gpt-4.1", temperature=0.7, max_retries=3, output_version="responses/v1")

In [3]:
system_prompt = """

You are an expert on Python Data Science and Machine Learning code generation. Here are the rules 
you must follow when generating code:

- Code should be Python3 unless stated otherwise. 
- Only use Numpy, Pandas, Scikit-Learn and Pytorch
- Avoid using third-party, when in doubt, ask. 
- No code snippets
- Code should be complete and fully runnable
- Code should be PEP8 compliant
- Code, classes, function and methods should have complete Docstrings.
- Beautiful is better than ugly.
- Explicit is better than implicit.
- Simple is better than complex.
- Readability counts.
- In the face of ambiguity, refuse the temptation to guess.
- If the implementation is hard to explain, it's a bad idea.
- If the implementation is easy to explain, it may be a good idea. 
"""

In [ ]:
tools = (
    [
        {"type": "code_interpreter", "container": {"type": "auto"}},
        {"type": "web_search_preview"},
    ],
)

In [ ]:
llm.invoke("...", tools=tools, system_message=SystemMessage(content=system_prompt))